This Note is using english/vander method

# Libraries

In [ ]:
!pip install Sastrawi
!pip install clean-text
!pip install deep-translator

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from cleantext import clean
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import sklearn

In [ ]:
# !pip freeze > requirements.txt

# Get Data from ES and prepare DF

In [ ]:
import requests
import json

In [ ]:
elasticsearch_url = 'https://es-1.datanusantara.com:9201'
username = 'adminkibana'
password = 'kibanaPASS!@#2019'
index_endpoint = 'news-crawl-production-v2'

In [ ]:
query = {
  "_source": ["target", "title","description_content"],
  "size": 600,
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "bool": {
            "should": [


              {
                "match_phrase": {
                  "target": "banggai"
                }
              }
            ],
            "minimum_should_match": 1


          }
        }
      ]
    }
  }
}
search_url = f"{elasticsearch_url}/{index_endpoint}/_search"
response = requests.get(search_url, json=query, auth=(username, password))

search_url

'https://es-1.datanusantara.com:9201/news-crawl-production-v2/_search'

In [ ]:
search_results = response.json()
# print(json.dumps(search_results, indent=2))
print(len(search_results['hits']['hits']))

600


In [ ]:
# Parse the response to extract relevant data
search_results = response.json()

# Process the data and create a DataFrame
data = []
for hit in search_results.get("hits", {}).get("hits", []):
    target = hit["_source"]["target"]
    title = hit["_source"]["title"]
    text = hit["_source"]["description_content"]

    data.append({"target": target,  "title": title, "text": text})

# Create a DataFrame
df = pd.DataFrame(data)
print(df.head())

    target                                              title  \
0  banggai  Mobil Sim Keliling Polres Banggai Meriahkan Pa...   
1  banggai  Film Horor “Piara Mayat”,  Kreativitas Anak Mu...   
2  banggai  Festival Malabot Tumbe 2023 Kembali Meriahkan ...   
3  banggai  Rakor Kampanye Damai Pemilu 2024, Polres Bangg...   
4  banggai  Bawaslu Banggai Perkuat Koordinasi dengan Panw...   

                                                text  
0  KABAR LUWUK –  Suasana Pasar Murah bersubsidi ...  
1  KABAR LUWUK  –  Film Horor“Piara Mayat”,Kreati...  
2  KABAR LUWUK  –Festival Malabot Tumbe 2023Kemba...  
3  “Kerja Cepat dan Terkoordinir dalam Penerbitan...  
4  KABAR LUWUK  –Bawaslu BanggaiPerkuat Koordinas...  


In [ ]:
# Drop Unnamed column
df.drop(df.columns[df.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

In [ ]:
df.head()

,target,title,text
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...


# Preprocessing

## Delete Duplicates

In [ ]:
df.duplicated(subset='title').sum()


14

In [ ]:
df = df.drop_duplicates(subset='title')

## Deleting Blank (Nan) Rows

In [ ]:
df = df[df['text'] != '']

In [ ]:
len(df)

579

In [ ]:
#row_index = 127  # Index of the row you want to print (Python indexing starts from 0)
#columns_to_print = ['title', 'text']  # List of column names to display

# Use .loc[] to select the row by its index and specify the columns to display
#row_data = df.loc[row_index, columns_to_print]

#print(row_data)

## Translate to English

In [ ]:
df = df.drop(columns=['lwr'])

In [ ]:
df.head()

,target,title,text
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...


In [ ]:
from deep_translator import GoogleTranslator

def en_translate (eng):
  eng = GoogleTranslator(source='id', target='en').translate(eng)
  return eng

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah di-translate
df['eng'] = en_translate(df['text'])
df['eng']

NotValidPayload: 0      KABAR LUWUK –  Suasana Pasar Murah bersubsidi ...
1      KABAR LUWUK  –  Film Horor“Piara Mayat”,Kreati...
2      KABAR LUWUK  –Festival Malabot Tumbe 2023Kemba...
3      “Kerja Cepat dan Terkoordinir dalam Penerbitan...
4      KABAR LUWUK  –Bawaslu BanggaiPerkuat Koordinas...
                             ...                        
594    BANGGAI RAYA-Satu gudang bahan aluminium di Ja...
595    Laporan Wartawan TribunPalu.com, Asnawi Zikri ...
596    BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...
598    Laporan Wartawan TribunPalu.com, Asnawi Zikri ...
599    Laporan Wartawan TribunPalu.com, Asnawi Zikri ...
Name: text, Length: 579, dtype: object --> text must be a valid text with maximum 5000 character,otherwise it cannot be translated

In [ ]:
# Define the translation function
def en_translate(eng):
    eng = GoogleTranslator(source='id', target='en').translate(eng)
    return eng

# Apply the translation function to the text at index 1
translated_text = en_translate(df['text'][1])

print(translated_text)

KABAR LUWUK – Horror film “Piara Corpse”, Creativity of Banggai Young People Makes a Footprint in the World of Cinematography. Banggai Regency, Central Sulawesi, is back in the spotlight with the latest original film produced by the creativity of its young people. Titled "Piara Corpse," this film was produced by Tosca Creativepreneur and is planned to be broadcast in mid-January. Tuesday 12/12/2023. Promotional process This film has started by releasing a video trailer via the official Instagram account @filmpiaramayat on February 11 2023, as well as the social media platforms of the crew involved in the production. The film producer, Iqbal Mustatim, explained that launching the video trailer is an important step in the process of making the film, indicating that the film will be broadcast soon. Previously, the Piara Mayat team had gone through various stages of pre-production and production, including script writing, casting, rehearsals and shooting. Tosca Creativepreneur collaborated

In [ ]:
from deep_translator import GoogleTranslator

text = df['text'][0]
print("og: ", text)
translated = GoogleTranslator(source='id', target='en').translate(text)
print("translated: ", translated)
back_translated = GoogleTranslator(source='en', target='id').translate(translated)
print("back translated: ", back_translated)

og:  KABAR LUWUK –  Suasana Pasar Murah bersubsidi di Kabupaten Banggai menjadi lebih semarak dengan kehadiran Satuan Lalulintas Polres Banggai yang membawa Mobil Sim Keliling. Senin 11/12/2023.Keberadaan ini bukan hanya sekadar pelayanan, melainkan juga menandai sinergi antara Polres Banggai dan Kejaksaan Negeri Banggai yang terjalin kuat.Kepala Satuan Lalu Lintas Polres Banggai, AKP I Made Bagus Aditya,melalui Kanit Regiden, Ipda Suparjan, memimpin stand pameran Sim Keliling di Pasar Murah bersubsidi.Suparjan menjelaskan bahwa kegiatan ini merupakan upaya konkret dalam mendekatkan diri kepada masyarakat, terutama dalam pengurusan perpanjangan Surat Izin Mengemudi (SIM).Masyarakat Kabupaten Banggai dapat dengan mudah mengurus perpanjangan SIM tanpa harus repot datang ke Kantor Satpas Polres Banggai dihalimun.Cukup dengan mendatangi Mobil Sim Keliling yang telah disediakan oleh tim, pelayanan pun siap dilayani. Ujarnya.Dalam suasana yang ramai, Ipda Suparjan menekankan bahwa kehadiran 

## Case Folding

In [ ]:
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah dicasefolding
df['lwr'] = df['text'].apply(clean_lower)
df['lwr']

In [ ]:
df.head()

In [ ]:
print(df['text'][30])
print("_______________________________________________________________________________________________________________")
print(df['lwr'][30])

KABAR LUWUK  –Dinas Sosial BanggaiSegera Salurkan 2.42 MiliarProgram Gercep Gaskan Berdaya2023.Dinas Sosial Kabupaten Banggaimenerima bantuan sebesar 2.42 miliar rupiah dalam program Gerak Cepat Pengentasan Kemiskinan Berbasis Pemberdayaan Masyarakat (GERCEP GASKAN BERDAYA).Bantuan ini ditujukan untuk 242 keluarga penerima manfaat di lima kecamatan, memberikan harapan bagi warga di Kecamatan Luwuk (35 KPM), Luwuk Selatan (28 KPM), Pagimana (95 KPM), Bunta (47 KPM), dan Nuhon (37 KPM). Selasa 28/11/2023.Kepala Dinas Sosial Kabupaten Banggai, Irpan Poma melalui Kepala Bidang Fakir Miskis,Ronald Putje,menyampaikan kepada media bahwa dana bantuan ini merupakan sumbangan langsung dari Pemerintah Provinsi Sulawesi Tengah.Dinas Sosial Kabupaten Banggai bertanggung jawab dalam memfasilitasi distribusi bantuan dan ikut serta dalam pengawasan ketat agar dana tersebut benar-benar sampai kepada penerima manfaat yang tepat.Data yang tersedia menunjukkan bahwa 242 Keluarga Penerima Manfaat yang terd

## Cleansing

### Remove Numbers

In [ ]:
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

In [ ]:
df['clean_number'] = df['lwr'].apply(remove_numbers)
df['clean_number']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe kembali m...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
594    banggai raya-satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu.com, asnawi zikri ...
596    banggai raya-lebih  pebulutangkis dari kabupat...
598    laporan wartawan tribunpalu.com, asnawi zikri ...
599    laporan wartawan tribunpalu.com, asnawi zikri ...
Name: clean_number, Length: 579, dtype: object

In [ ]:
df.tail()

,target,title,text,lwr,clean_number
594,banggai,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...
595,banggai,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
596,banggai,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...
598,banggai,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
599,banggai,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."


### Remove Punctuation

In [ ]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

In [ ]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
df['clean_punct'] = df['clean_number'].apply(clean_punct)
df['clean_punct']

0      kabar luwuk    suasana pasar murah bersubsidi ...
1      kabar luwuk     film horor piara mayat kreativ...
2      kabar luwuk   festival malabot tumbe kembali m...
3       kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk   bawaslu banggaiperkuat koordinas...
                             ...                        
594    banggai raya satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu com asnawi zikri t...
596    banggai raya lebih  pebulutangkis dari kabupat...
598    laporan wartawan tribunpalu com asnawi zikri t...
599    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_punct, Length: 579, dtype: object

In [ ]:
df.head()


,target,title,text,lwr,clean_number,clean_punct
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...


### Delete whitespaces

In [ ]:
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
df['clean_double_ws'] = df['clean_punct'].apply(normalize_whitespace)
df['clean_double_ws']

0      kabar luwuk suasana pasar murah bersubsidi di ...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe kembali mer...
3      kerja cepat dan terkoordinir dalam penerbitan ...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
594    banggai raya satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu com asnawi zikri t...
596    banggai raya lebih pebulutangkis dari kabupate...
598    laporan wartawan tribunpalu com asnawi zikri t...
599    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_double_ws, Length: 579, dtype: object

In [ ]:
df.tail()

,target,title,text,lwr,clean_number,clean_punct,clean_double_ws
594,banggai,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...
595,banggai,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
596,banggai,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupate...
598,banggai,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
599,banggai,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...


## Stopword Removal

### Get custom stopwords

In [ ]:
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

In [ ]:
url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

In [ ]:
df_stopword.info()

In [ ]:
df_stopword

In [ ]:
df.head()

In [ ]:
custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

### Removal

In [ ]:
# Prepare stopword remover
#factory = StopWordRemoverFactory()
#stopwords = factory.get_stop_words() + custom_stopword
#print(stopwords)

In [ ]:
# using only sastrawy
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2

In [ ]:
len(stopwords)

946

In [ ]:
dictionary = ArrayDictionary(stopwords)
stopword = StopWordRemover(dictionary)


In [ ]:
stopwords

In [ ]:
def filtering(text):
  stop = stopword.remove(text)
  return stop

In [ ]:
df.head()

In [ ]:
#df.drop(['filtered'], axis=1, inplace=True)

In [ ]:
df['filtered'] = df['clean_double_ws'].apply(filtering)
df['filtered']

0      luwuk suasana pasar murah bersubsidi semarak k...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir penerbitan sttp luwuk...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
594    raya gudang bahan aluminium jalan teuku umar k...
595    wartawan asnawi tribunpalu com banggai membuka...
596    raya pebulutangkis kabupatenkota sulawesi berl...
598    wartawan asnawi tribunpalu com banggai pria ne...
599    wartawan asnawi tribunpalu com banggai gudang ...
Name: filtered, Length: 579, dtype: object

In [ ]:
df.head()

In [ ]:
len(df)

579

In [ ]:
print(df['clean_double_ws'][55])
print(df['filtered'][55])

## Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [ ]:
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()

In [ ]:
def stem(text):
  return stemmer.stem(text)

In [ ]:
df.head()

In [ ]:
df['stemmed'] = df['filtered'].apply(stem)
df['stemmed']

0      luwuk suasana pasar murah subsidi semarak hadi...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir terbit sttp luwuk rak...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
594    raya gudang bahan aluminium jalan teuku umar k...
595    wartawan asnawi tribunpalu com bangga buka aca...
596    raya pebulutangkis kabupatenkota sulawesi laga...
598    wartawan asnawi tribunpalu com bangga pria nek...
599    wartawan asnawi tribunpalu com bangga gudang b...
Name: stemmed, Length: 579, dtype: object

In [ ]:
df.head()

## Export df to csv (to cutting time if we want to stepback process)

In [ ]:
df.to_csv("stemmed_words_sentiment1.csv", index=False)

## Importing df rom csv (cutting time if we want to stepback process)

In [ ]:
# to skip stemming process
stemmed_url= 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai_Sentiment/main/stemmed_words_sentiment1.csv'

df = pd.read_csv(stemmed_url,encoding='utf-8')

df = df.astype(str)

In [ ]:
df.head()


In [ ]:
len(df)

579

In [ ]:
print(df['filtered'][55])
print(df['stemmed'][55])

permakita untikaluwuk rayakan natal perdana meriah persatuan mahasiswa kristen universitas tompotika permakitasukses menggelar perayaan natal perdana tanggal desember lantai fakultas kesehatan masyarakat untika kegiatan menjadi momen bersatu mahasiswa kristen lingkupuntika luwukuntuk merayakan kelahiran yesus kristus wakil rektor iii dr hamdin husin m si perwakilan pimpinan birokrasi universitas hadir mengapresiasi kegiatan sambutannya beliau harapannya kegiatan terus tahun pentingnya kegiatan positif lingkungan universitas perayaan natal perdana untika memberikan dampak positif seluruh mahasiswa kristen dr hamdin husin m si ketua panitia andre apriolio kaloke tema perayaan diangkat kemuliaan allah damai sejahterah bumi lucas dalam sambutannya andre terima kasih semua panitia mahasiswa telah berkontribusi mendukung kegiatan mewakili teman teman panitia mengucapkan terima kasih seluruh donatur kampus pihak serta panitia telah berkontribusi penuh meluangkan tenaga waktu kegiatan sukses t

In [ ]:
print(len(df['filtered'][55]))
print(len(df['stemmed'][55]))

1306
1108


## Tokenizing

In [ ]:
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [ ]:
df.head()

In [ ]:
df['tokenized'] = df['stemmed'].apply(tokenize)
df['tokenized']

0      [luwuk, suasana, pasar, murah, subsidi, semara...
1      [luwuk, film, horor, piara, mayat, kreativitas...
2      [luwuk, festival, malabot, tumbe, meriahkankab...
3      [kerja, cepat, terkoordinir, terbit, sttp, luw...
4      [luwuk, bawaslu, banggaiperkuat, koordinasi, d...
                             ...                        
574    [raya, gudang, bahan, aluminium, jalan, teuku,...
575    [wartawan, asnawi, tribunpalu, com, bangga, bu...
576    [raya, pebulutangkis, kabupatenkota, sulawesi,...
577    [wartawan, asnawi, tribunpalu, com, bangga, pr...
578    [wartawan, asnawi, tribunpalu, com, bangga, gu...
Name: tokenized, Length: 579, dtype: object

In [ ]:
print(df['filtered'][55])
print(df['tokenized'][55])

permakita untikaluwuk rayakan natal perdana meriah persatuan mahasiswa kristen universitas tompotika permakitasukses menggelar perayaan natal perdana tanggal desember lantai fakultas kesehatan masyarakat untika kegiatan menjadi momen bersatu mahasiswa kristen lingkupuntika luwukuntuk merayakan kelahiran yesus kristus wakil rektor iii dr hamdin husin m si perwakilan pimpinan birokrasi universitas hadir mengapresiasi kegiatan sambutannya beliau harapannya kegiatan terus tahun pentingnya kegiatan positif lingkungan universitas perayaan natal perdana untika memberikan dampak positif seluruh mahasiswa kristen dr hamdin husin m si ketua panitia andre apriolio kaloke tema perayaan diangkat kemuliaan allah damai sejahterah bumi lucas dalam sambutannya andre terima kasih semua panitia mahasiswa telah berkontribusi mendukung kegiatan mewakili teman teman panitia mengucapkan terima kasih seluruh donatur kampus pihak serta panitia telah berkontribusi penuh meluangkan tenaga waktu kegiatan sukses t

In [ ]:
print(len(df['stemmed'][55]))
print(len(df['tokenized'][55]))

1108
162


##Second Stopwords

In [ ]:
df.head()

In [ ]:
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
df['filtered2'] = df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

# Display the updated DataFrame
print(df.head())

In [ ]:
#df.drop(columns=['tokenized'], inplace=True)

In [ ]:
len(stopwords)

946

In [ ]:
df.head()

,target,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized,filtered2
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...,luwuk suasana pasar murah bersubsidi semarak k...,luwuk suasana pasar murah subsidi semarak hadi...,"[luwuk, suasana, pasar, murah, subsidi, semara...","[suasana, pasar, murah, subsidi, semarak, hadi..."
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...,luwuk film horor piara mayat kreativitasanak m...,luwuk film horor piara mayat kreativitasanak m...,"[luwuk, film, horor, piara, mayat, kreativitas...","[film, horor, piara, mayat, kreativitasanak, m..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...,luwuk festival malabot tumbe meriahkankabupate...,luwuk festival malabot tumbe meriahkankabupate...,"[luwuk, festival, malabot, tumbe, meriahkankab...","[festival, malabot, tumbe, meriahkankabupaten,..."
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...,kerja cepat terkoordinir penerbitan sttp luwuk...,kerja cepat terkoordinir terbit sttp luwuk rak...,"[kerja, cepat, terkoordinir, terbit, sttp, luw...","[kerja, cepat, terkoordinir, terbit, sttp, rak..."
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,luwuk bawaslu banggaiperkuat koordinasi dengan...,luwuk bawaslu banggaiperkuat koordinasi dengan...,"[luwuk, bawaslu, banggaiperkuat, koordinasi, d...","[bawaslu, banggaiperkuat, koordinasi, denganpa..."


In [ ]:
print(df['tokenized'][55])
print(df['filtered2'][55])

# Text Classification

In [ ]:
# For manual Review
#import random

# Set the number of random samples you want to select
#num_samples = 100  # Adjust this number as needed

# Select random samples from the DataFrame
#random_indices = random.sample(range(len(df)), num_samples)
#random_data = df.iloc[random_indices][["title", "text"]]

# Save random data to a CSV file
#random_data.to_csv("random_data.csv")


In [ ]:
df.info()

## Preprocessing (Text Classification)

In [ ]:
df.head()

In [ ]:
# Spliting database to train_set and test_set
from sklearn.model_selection import train_test_split

X = df['filtered2']
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [ ]:
X_train.head()

95     [lembaga, jamin, mutu, kembang, didik, lpmpp, ...
104    [program, studi, prodi, manajemen, fakultas, e...
66     [juni, inflasi, year, on, year, yoy, kota, cat...
0      [warga, jaring, utara, nama, josua, bangun, in...
141    [anggota, satlantaspolres, banggaimelaksanakan...
Name: filtered2, dtype: object

In [ ]:
print(y_test.unique())


#df['Topic'].nunique()

In [ ]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

# Fit Vectorizer to the data (build a vocab, count the number of words..)
# and transfrom the original text message to ---> Vector

from itertools import chain

# Assuming X_train is a list of lists or a list of tokens
flattened_X_train = list(chain.from_iterable(X_train))
flattened_X_test = list(chain.from_iterable(X_test))

# Assuming X_train is a list of tokens
X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

X_train_counts = count_vect.fit_transform(X_train)


In [ ]:
X_train_counts

<99x3291 sparse matrix of type '<class 'numpy.int64'>'
	with 8942 stored elements in Compressed Sparse Row format>

In [ ]:
X_train.shape

(99,)

In [ ]:
X_train_counts.shape

(99, 3291)

In [ ]:
# TFIDFVectorizer and LinearSVC with pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train, y_train)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
# Checking performance using confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(f"Accuracy {metrics.accuracy_score(y_test, predictions)}")

[[ 4  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0]
 [ 0  0 16  0  1  1  0  0]
 [ 1  0  0  4  0  0  0  0]
 [ 3  0  2  0  5  1  0  0]
 [ 0  0  0  0  0  4  0  0]
 [ 0  0  2  0  0  0  0  0]
 [ 0  2  1  0  0  0  0  0]]
                                          precision    recall  f1-score   support

                           Ekonomi Lokal       0.50      1.00      0.67         4
Infrastruktur, Pembangunan dan Perumahan       0.60      1.00      0.75         3
             Keamanan dan Layanan Publik       0.76      0.89      0.82        18
                        Kesehatan Publik       1.00      0.80      0.89         5
        Lingkungan dan Partisipasi Warga       0.83      0.45      0.59        11
                              Pendidikan       0.67      1.00      0.80         4
                                 Politik       0.00      0.00      0.00         2
                            Transportasi       0.00      0.00      0.00         3

                                accuracy   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Input Data

In [ ]:
# Input
text_url = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/input_text.csv'

# Read CSV file with 'latin-1' encoding
input_df = pd.read_csv(text_url, encoding='utf-8')

# Convert all columns to string
input_df = input_df.astype(str)

In [ ]:
input_df.head()

## Preprocessing

In [ ]:
# Delete Duplicates
input_df.duplicated(subset='title').sum()
input_df = input_df.drop_duplicates(subset='title')

# Delete Blank text (Nan) Rows
input_df = input_df[input_df['text'] != '']

# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

input_df['lwr'] = input_df['text'].apply(clean_lower)
input_df['lwr']


# Remove Numbers
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

input_df['clean_number'] = input_df['lwr'].apply(remove_numbers)
input_df['clean_number']


# Remove Punctuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

input_df['clean_punct'] = input_df['clean_number'].apply(clean_punct)
input_df['clean_punct']


# Delete Whitespaces
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

input_df['clean_double_ws'] = input_df['clean_punct'].apply(normalize_whitespace)
input_df['clean_double_ws']


In [ ]:
len(input_df)

In [ ]:
input_df.head()

In [ ]:
# Adding Stopwords
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2


# 1st Stopwords Removal
def filtering(text):
  stop = stopword.remove(text)
  return stop

input_df['filtered'] = input_df['clean_double_ws'].apply(filtering)
input_df['filtered']


# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def stem(text):
  return stemmer.stem(text)

input_df['stemmed'] = input_df['filtered'].apply(stem)
input_df['stemmed']


# Tokenizing
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

input_df['tokenized'] = input_df['stemmed'].apply(tokenize)
input_df['tokenized']


# 2nd Stopwords Removal
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
input_df['filtered2'] = input_df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

In [ ]:
input_df.head()

In [ ]:
#input_df.drop(columns=['predictions'], inplace=True)

In [ ]:
# Convert tokenized text to string representation
#text_strings = [' '.join(doc) for doc in df['tokenized']]
input_df['text2'] = input_df['tokenized'].apply(lambda x: ' '.join(x))


In [ ]:
input_df.head()

# Predicting Input Data

In [ ]:

# Assuming 'selected_features' is a list of feature names used during model training
X_input = input_df['text2']

# 3. Predict using the model
predictions = text_clf.predict(X_input)

# 4. Add predictions to a new column in the DataFrame
input_df['topic'] = predictions

In [ ]:
input_df.head()



```
# This is formatted as code
```

## Convert input_df to csv

In [ ]:
# Assuming 'df' is your existing DataFrame and 'columns_to_keep' is a list of column names you want to keep
columns_to_keep = ['title', 'text', 'topic']  # Adjust with your desired column names

# Create a new DataFrame with only the selected columns
final_text = input_df[columns_to_keep].copy()

final_text.to_csv('final_text.csv', index=False)

In [ ]:
import pickle
#pickle.dump(model, open('model_name_you_want_to_save_as.sav', 'wb'))
pickle.dump(text_clf, open('text_classifier.sav', 'wb'))